The system is simplified as a 3-node system:
$$\frac{dm}{dt}=\alpha \frac{a_f}{a_t}-m$$
$$\frac{dp}{dt}=\alpha_0g(x/c_d)m-p$$
$$\frac{dp_{nuc}}{dt}=p-p_{nuc}$$
<!--$$\frac{dV}{dt}=kV^{1/3}(x/c_d-V/V_c)$$-->
$$x=A(1+sin(\omega t))$$
$$g(n)=\frac{(ef-1)n+1}{((f-1)n+1)((e-1)n+1)}$$
$$a_f=\frac{1}{2}(a_t-p-1+\sqrt{(a_t-p-1)^2+4a_t})$$


The chemical reactions in this system are:
- $$\emptyset \overset{\alpha a_f/a_t}\longrightarrow M $$
- $$M\overset{M}\longrightarrow \emptyset$$
- $$\emptyset \overset{\alpha_0gM}\longrightarrow P$$
- $$P\overset{P}\longrightarrow P_{nuc}$$
- $$P_{nuc}\overset{P_{nuc}}\longrightarrow\emptyset$$


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D
# from scipy.integrate import solve_ivp
# from fenics import *

global alpha,at,alpha0,cd,e,f,A,omega,v,mnum,pnum,pnucnum
# Parameters
alpha=200
at=40
alpha0=1
cd=100
e=50
f=50
A=50
omega=np.pi/12
v=np.array([[1,-1,0,0,0],
                  [0,0,1,-1,0],
                  [0,0,0,1,-1]]) # Chemical counting matrix
mnum=100
pnum=500
pnucnum=500

# function drift returns f_i*P np.array(3)
def drift(t,m,p,p_nuc):
    # intermediate variables
    af=0.5*(at-p-1+np.sqrt((at-p-1)**2+4*at))
    s=A*(np.sin(omega*t))/cd
    g=((e*f-1)*s+1)/((f-1)*s+1)/((e-1)*s+1)
    # reaction rates
    r=np.zeros(5)
    r[0]=alpha*af/at
    r[1]=m
    r[2]=alpha0*g*m
    r[3]=p
    r[4]=p_nuc
    # drift term
    drift=np.zeros(3)
    for i in range(3):
        for j in range(5):
            drift[i]+=v[i,j]*r[j]
    return drift
# function diffusion returns D_ij*P np.array(3,3)
def diffusion(t,m,p,p_nuc):
    # intermediate variables
    af=0.5*(at-p-1+np.sqrt((at-p-1)**2+4*at))
    s=A*(np.sin(omega*t))/cd
    g=((e*f-1)*s+1)/((f-1)*s+1)/((e-1)*s+1)
    # reaction rates
    r=np.zeros(5)
    r[0]=alpha*af/at
    r[1]=m
    r[2]=alpha0*g*m
    r[3]=p
    r[4]=p_nuc
    # diffusion matrix
    D=np.zeros((3,3))
    for i in range(3):
        for j in range(3):
            D[i,j]=0
            for k in range(5):
                D[i,j]+=v[i,k]*v[j,k]*r[k]
    return D

def fokker_planck(t, P):
    # 更新增量
    dpdt = np.zeros((mnum, pnum, pnucnum))
    for i in range(mnum):
        for j in range(pnum):
            for k in range(pnucnum):
                dpdt[i, j, k] = -(drift(t, i, j, k)[0] * P[i, j, k] - drift(t, i-1, j, k)[0] * P[i-1, j, k])          \
                    -(drift(t, i, j, k)[1] * P[i, j, k] - drift(t, i, j-1, k)[1] * P[i, j-1, k])                  \
                    -(drift(t, i, j, k)[2] * P[i, j, k] - drift(t, i, j, k-1)[2] * P[i, j, k-1])                  
        
                dpdt[i, j, k] += 0.5*(diffusion(t, i, j, k)[0, 0] * P[i, j, k] - 2*diffusion(t, i-1, j, k)[0, 0] * P[i-1, j, k] + diffusion(t, i-2, j, k)[0, 0] * P[i-2, j, k]) \
                    +0.5*(diffusion(t, i, j, k)[1, 1] * P[i, j, k] - 2*diffusion(t, i, j-1, k)[1, 1] * P[i, j-1, k] + diffusion(t, i, j-2, k)[1, 1] * P[i, j-2, k]) \
                    +0.5*(diffusion(t, i, j, k)[2, 2] * P[i, j, k] - 2*diffusion(t, i, j, k-1)[2, 2] * P[i, j, k-1] + diffusion(t, i, j, k-2)[2, 2] * P[i, j, k-2]) \
                
                dpdt[i, j, k] += 0.5*(diffusion(t, i, j, k)[0, 1] * P[i, j, k] - diffusion(t, i-1, j, k)[0, 1] * P[i-1, j, k] - diffusion(t, i, j-1, k) * P[i, j-1, k] + diffusion(t, i-1, j-1, k) * P[i-1, j-1, k]) \
                    +0.5*(diffusion(t, i, j, k)[0, 2] * P[i, j, k] - diffusion(t, i-1, j, k)[0, 2] * P[i-1, j, k] - diffusion(t, i, j, k-1) * P[i, j, k-1] + diffusion(t, i-1, j, k-1) * P[i-1, j, k-1]) \
                    +0.5*(diffusion(t, i, j, k)[1, 2] * P[i, j, k] - diffusion(t, i, j-1, k)[1, 2] * P[i, j-1, k] - diffusion(t, i, j, k-1) * P[i, j, k-1] + diffusion(t, i, j-1, k-1) * P[i, j-1, k-1]) \
                    +0.5*(diffusion(t, i, j, k)[1, 0] * P[i, j, k] - diffusion(t, i, j-1, k)[1, 0] * P[i, j-1, k] - diffusion(t, i-1, j, k) * P[i-1, j, k] + diffusion(t, i-1, j-1, k) * P[i-1, j-1, k]) \
                    +0.5*(diffusion(t, i, j, k)[2, 0] * P[i, j, k] - diffusion(t, i, j, k-1)[2, 0] * P[i, j, k-1] - diffusion(t, i-1, j, k) * P[i-1, j, k] + diffusion(t, i-1, j, k-1) * P[i-1, j, k-1]) \
                    +0.5*(diffusion(t, i, j, k)[2, 1] * P[i, j, k] - diffusion(t, i, j, k-1)[2, 1] * P[i, j, k-1] - diffusion(t, i, j-1, k) * P[i, j-1, k] + diffusion(t, i, j-1, k-1) * P[i, j-1, k-1])
                
    return dpdt
simu_time=10
dt=0.1
timestep=simu_time*10

shape = (100, 100, 500, 500)
dtype = np.float64
filename = "large_array_0524_1.dat"
trajectory = np.memmap(filename, dtype=dtype, mode='w+', shape=shape)

# 使用 fp 作为数组
trajectory[:] = 0.0
# trajectory=np.zeros((timestep,mnum,pnum,pnucnum))               

# initial condition
# trajectory[0]=np.zeros((mnum, pnum, pnucnum))
trajectory[0][80][50][30]=1


for i in range(timestep-1):
    trajectory[i+1]=trajectory[i]+dt*fokker_planck(i*dt,trajectory[i])            
               
trajec_mpnuc=sum(trajectory,axis=3)  # 计算每个时间步的总概率 

# 创建网格用于绘图
x = np.arange(mnum)  # 第一个变量维度的网格点
y = np.arange(pnucnum)  # 第二个变量维度的网格点
X, Y = np.meshgrid(x, y)
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

def init():
    # 设置初始绘图
    ax.set_xlabel('Dimension 1')
    ax.set_ylabel('Dimension 2')
    ax.set_zlabel('Probability Density')
    ax.set_title('Probability Distribution Evolution')
    return []

def update(frame):
    # 更新每帧的绘图
    ax.clear()
    ax.set_xlabel('Dimension 1')
    ax.set_ylabel('Dimension 2')
    ax.set_zlabel('Probability Density')
    ax.set_title(f'Time Step: {frame}')
    Z = trajec_mpnuc[frame, :, :]  # 获取当前时间步的概率密度
    # 归一化以确保颜色映射的一致性
    Z_normalized = (Z - np.min(Z)) / (np.max(Z) - np.min(Z))
    # 绘制三维箭头表示概率密度方向和大小
    # 这是一个简化的绘图方法，实际应用中可根据需要调整
    ax.quiver(X[::10, ::10], Y[::10, ::10], np.zeros_like(X[::10, ::10]), 
              np.zeros_like(X[::10, ::10]), np.zeros_like(Y[::10, ::10]), Z_normalized[::10, ::10], 
              cmap='viridis', norm=plt.Normalize(0, 1))
    # 可以使用其他绘图方法，如 surf
    # surf = ax.plot_surface(X, Y, Z, cmap='viridis', edgecolor='none')
    # fig.colorbar(surf, shrink=0.5, aspect=10)
    return [ax]
# 创建动画
ani = FuncAnimation(fig, update, frames=10000, init_func=init, interval=200, blit=False)
plt.tight_layout()
plt.show()
# 保存动画（可选）
ani.save('prob_evolution.gif', writer='pillow', fps=30)

ValueError: setting an array element with a sequence.